In [2]:
from google.colab import files
uploaded = files.upload()

Saving lr_model.joblib to lr_model.joblib
Saving rf_model.joblib to rf_model.joblib
Saving sc_model.joblib to sc_model.joblib
Saving svc_model.joblib to svc_model.joblib
Saving vc_model.joblib to vc_model.joblib
Saving xgb_model.joblib to xgb_model.joblib


In [3]:
import joblib
import pandas as pd
import os
import random

### Create test customer data

In [23]:
customer_churn_data = {
    'customerID': ['0', '1', '2', '3', '4'],
    'gender': ['Female', 'Male', 'Male', 'Male', 'Female'],
    'SeniorCitizen': [0, 0, 0, 0, 0],
    'Partner': ['Yes', 'No', 'No', 'No', 'No'],
    'Dependents': ['No', 'No', 'Yes', 'No', 'No'],
    'tenure': [1, 34, 2, 45, 2],
    'PhoneService': ['No', 'Yes', 'Yes', 'No', 'Yes'],
    'MultipleLines': ['No phone service', 'No', 'No', 'Yes', 'No'],
    'InternetService': ['DSL', 'DSL', 'DSL', 'DSL', 'Fiber optic'],
    'OnlineSecurity': ['No', 'Yes', 'Yes', 'Yes', 'No'],
    'OnlineBackup': ['Yes', 'No', 'No', 'No', 'No'],
    'DeviceProtection': ['No', 'No', 'No', 'Yes', 'No'],
    'TechSupport': ['No', 'Yes', 'No', 'Yes', 'No'],
    'StreamingTV': ['No', 'No', 'No', 'No', 'No'],
    'StreamingMovies': ['No', 'No', 'No', 'Yes', 'No'],
    'Contract': ['Month-to-month', 'One year', 'Month-to-month', 'One year', 'Month-to-month'],
    'PaperlessBilling': ['Yes', 'No', 'Yes', 'No', 'Yes'],
    'PaymentMethod': ['Electronic check', 'Mailed check', 'Mailed check', 'Bank transfer (automatic)', 'Electronic check'],
    'MonthlyCharges': [29.85, 56.95, 53.85, 42.30, 70.70],
    'TotalCharges': [29.85, 1889.50, 108.15, 1840.75, 151.65],
    'Churn': ['No', 'No', 'Yes', 'No', 'Yes']
}

In [24]:
# shuffle the dataset
df = pd.DataFrame(customer_churn_data)
df = df.sample(frac=1).reset_index(drop=True)
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,3,Male,0,No,No,45,No,Yes,DSL,Yes,...,Yes,Yes,No,Yes,One year,No,Bank transfer (automatic),42.30,1840.75,No
1,2,Male,0,No,Yes,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
2,1,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,Yes,No,No,One year,No,Mailed check,56.95,1889.50,No
3,4,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
4,0,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No


#### Remove unneeded features

In [25]:
df["Partner"]=df["Partner"].replace({"Yes":1,"No":0})
df["Dependents"]=df["Dependents"].replace({"Yes":1,"No":0})
df["PhoneService"]=df["PhoneService"].replace({"Yes":1,"No":0})
df["MultipleLines"]=df["MultipleLines"].replace({"Yes":1,"No":0,"No phone service":-1})
df["InternetService"]=df["InternetService"].replace({"DSL":1,"Fiber optic":2,"No":0})
df["OnlineSecurity"]=df["OnlineSecurity"].replace({"Yes":1,"No":0,"No internet service":-1})
df["OnlineBackup"]=df["OnlineBackup"].replace({"Yes":1,"No":0,"No internet service":-1})
df["DeviceProtection"]=df["DeviceProtection"].replace({"Yes":1,"No":0,"No internet service":-1})
df["TechSupport"]=df["TechSupport"].replace({"Yes":1,"No":0,"No internet service":-1})
df["StreamingTV"]=df["StreamingTV"].replace({"Yes":1,"No":0,"No internet service":-1})
df["StreamingMovies"]=df["StreamingMovies"].replace({"Yes":1,"No":0,"No internet service":-1})
df["PaperlessBilling"]=df["PaperlessBilling"].replace({"Yes":1,"No":0})
df["Churn"]=df["Churn"].replace({"Yes":1,"No":0})
df["PaymentMethod"]=df["PaymentMethod"].replace({"Electronic check":1,"Mailed check":2,"Bank transfer (automatic)":3,"Credit card (automatic)":4})
df["Contract"]=df["Contract"].replace({"Month-to-month":1,"One year":2,"Two year":3})

df.drop(columns = ["customerID", "TotalCharges", "MonthlyCharges", "tenure", "gender"], inplace=True)

df

,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Churn
0,0,0,0,0,1,1,1,0,1,1,0,1,2,0,3,0
1,0,0,1,1,0,1,1,0,0,0,0,0,1,1,2,1
2,0,0,0,1,0,1,1,0,0,1,0,0,2,0,2,0
3,0,0,0,1,0,2,0,0,0,0,0,0,1,1,1,1
4,0,1,0,0,-1,1,0,1,0,0,0,0,1,1,1,0


### Show comparison between expectation and prediction

In [26]:
def display_result(model_name, prediction):
    print("\n\n", model_name)
    result_df = pd.DataFrame()
    result_df["expected"] = df["Churn"].apply(lambda x: "Yes" if x == 1 else "No")
    result_df["prediction"] = ["Yes" if pred == 1 else "No" for pred in prediction]

    display(result_df)

### Load all models and predict customer churn

In [28]:
dir = "/content/saved_models"
saved_models = os.listdir(dir)

models = []

for model_name in saved_models:
    model = joblib.load(os.path.join(dir, model_name))
    name = model_name.split(".")[0]
    models.append(name)
    prediction = model.predict(df.drop("Churn", axis=1))
    display_result(name, prediction)



 vc_model


,expected,prediction
0,No,No
1,Yes,No
2,No,No
3,Yes,Yes
4,No,No




 svc_model


,expected,prediction
0,No,No
1,Yes,No
2,No,No
3,Yes,Yes
4,No,No




 lr_model


...

,expected,prediction
0,No,No
1,Yes,No
2,No,No
3,Yes,Yes
4,No,No




 sc_model


,expected,prediction
0,No,No
1,Yes,No
2,No,No
3,Yes,Yes
4,No,No




 xgb_model


,expected,prediction
0,No,No
1,Yes,No
2,No,No
3,Yes,Yes
4,No,No
